In [ ]:
import os
from dotenv import load_dotenv
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import (
    StorageContext,
    VectorStoreIndex,
    Settings,
    Document
)
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
import pandas as pd
from dataclasses import dataclass

In [ ]:
load_dotenv()

In [ ]:
# Settings.embed_model = CohereEmbedding(
#         cohere_api_key=os.getenv("COHERE_API_KEY"),
#         model_name="embed-english-v3.0",
#         input_type="search_document",
#     )

In [ ]:
Settings.embed_model = OpenAIEmbedding(
    api_key=os.getenv("OPENAI_API_KEY"), model="text-embedding-3-small", dimensions=1024
)

In [ ]:
# Create the sentence window node parser w/ default settings
# Code straight from llamaindex docs
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
    include_metadata=True,
)

In [ ]:
# Load data
data = pd.read_parquet("./data/parsed-dogmatics/dogmatics-text-w-meta-no-editors.parquet")

In [ ]:
data.head()

In [ ]:
# Change \n to a space in text column
data["text"] = data["text"].str.replace("\n", " ")

In [ ]:
@dataclass()
class Paragraph:
    text: str
    excursus: bool
    title: str
    group: str
    volume: str
    date: str
    biblScope: str

In [ ]:
Paragraphs = [Paragraph(*row) for row in data.itertuples(index=False)]

In [ ]:
Paragraphs[0]


In [ ]:
# Create llamaindex documents out of paragraphs, with metadata
docs = [
    Document(
        text=p.text,
        metadata={
            "excursus": p.excursus,
            "title": p.title,
            "group": p.group,
            "volume": p.volume,
            "date": p.date,
            "biblScope": p.biblScope,
        },
    )
    for p in Paragraphs if p.text is not None
]

In [ ]:
# Check the length of paragraphs vs docs
len(Paragraphs), len(docs)

In [ ]:
# Look into paragraphs with no text
# Given the POC nature of this particular work, 
# we'll ignore it for now and look in depth later
[p for p in Paragraphs if p.text is None]

In [ ]:
# Take a look at a single doc to check if well-formed
docs[0]

In [ ]:
# Create nodes with the SentenceWindowSplitter
nodes = node_parser.get_nodes_from_documents(docs, show_progress=True)

In [ ]:
nodes[0]

In [ ]:
# Create the Pinecone index
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
        name="dogmatics",
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        ),
    )

In [ ]:
pinecone_index = pc.Index("dogmatics")
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex(nodes, storage_context=storage_context, show_progress=True)